In [12]:
# !gzip -d "/home/pes1ug22am100/Documents/Research and Experimentation/NetworkAnalysis/equinix_nyc_data/equinix-nyc.dirA.20190117-130000.UTC.anon.pcap.gz"
# I unzipped this via terminal

# skip this now cuz files are already unzipped

gzip: /home/pes1ug22am100/Documents/Research and Experimentation/NetworkAnalysis/equinix_nyc_data/equinix-nyc.dirA.20190117-130000.UTC.anon.pcap.gz: No such file or directory


In [13]:
!tcpdump -r "/home/pes1ug22am100/Documents/Research and Experimentation/NetworkAnalysis/equinix_nyc_data/equinix-nyc.dirA.20190117-125910.UTC.anon.pcap" -w "/home/pes1ug22am100/Documents/Research and Experimentation/NetworkAnalysis/chonky_130000/chunked_file.pcap" -C 100
# this is to chunk into 100mb
# make sure to change directiry name
# I'm keeping the format as chonky_<fileID>

reading from file /home/pes1ug22am100/Documents/Research and Experimentation/NetworkAnalysis/equinix_nyc_data/equinix-nyc.dirA.20190117-125910.UTC.anon.pcap, link-type RAW (Raw IP), snapshot length 65536


In [14]:
import pyshark
import nest_asyncio
import csv

In [15]:
# Apply the nested asyncio patch cuz jupyter's weird
nest_asyncio.apply()

In [16]:
pcap_path = '/home/pes1ug22am100/Documents/Research and Experimentation/NetworkAnalysis/chonky_130000/chunked_file.pcap'

In [17]:
# Read the chunk
cap = pyshark.FileCapture(pcap_path)

for i, packet in enumerate(cap):
    try:
        print(f"Packet {i+1}:")
        print(f"  Timestamp: {packet.sniff_time}")
        print(f"  Source: {packet.ip.src}")
        print(f"  Destination: {packet.ip.dst}")
        print(f"  Protocol: {packet.highest_layer}")
        print(f"  Length: {packet.length}")
    except AttributeError:
        print(f"  Packet {i+1}: Missing IP information")
    if i >= 4:  # top 5 packets
        break


Packet 1:
  Timestamp: 2019-01-17 18:29:10.467105
  Source: 26.120.99.176
  Destination: 175.84.134.113
  Protocol: TCP
  Length: 1504
Packet 2:
  Timestamp: 2019-01-17 18:29:10.467106
  Source: 181.47.185.47
  Destination: 30.241.116.225
  Protocol: TCP
  Length: 56
Packet 3:
  Timestamp: 2019-01-17 18:29:10.467106
  Source: 169.85.223.179
  Destination: 136.127.65.205
  Protocol: TCP
  Length: 1484
Packet 4:
  Timestamp: 2019-01-17 18:29:10.467107
  Source: 169.85.223.179
  Destination: 136.127.65.205
  Protocol: TCP
  Length: 1484
Packet 5:
  Timestamp: 2019-01-17 18:29:10.467108
  Source: 42.27.128.236
  Destination: 180.211.115.195
  Protocol: _WS.SHORT
  Length: 1470


In [19]:
output_file = "/home/pes1ug22am100/Documents/Research and Experimentation/NetworkAnalysis/CSVdata/dirA.130000-packets.csv"
with open(output_file, mode="w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["Timestamp", "Source", "Destination", "Protocol", "Length"])
    
    for i, packet in enumerate(cap):
        try:
            writer.writerow([
                packet.sniff_time,
                packet.ip.src,
                packet.ip.dst,
                packet.highest_layer,
                packet.length,
            ])
        except AttributeError:
            # Handle packets that might not have IP or other fields
            continue

print(f"Packet details saved to {output_file}")